In [1]:
import matplotlib
import matplotlib.pyplot as plt

import jax
import jax.numpy as np
from jax.random import PRNGKey

import numpyro
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS, Predictive

import pandas as pd

from covid.models.SEIR_hierarchical import SEIR_hierarchical
import covid.util as util


Bad key "nbagg.transparent" on line 426 in
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.2.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key "animation.mencoder_path" on line 509 in
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.2.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key "animation.mencoder_args" on line 512 in
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle.
You probably need to get an update

# Run Inference

In [ ]:
#states = ['MA', 'NY', 'WA', 'AL']
states = ['MA', 'NY']
data, place_data = util.load_state_Xy(which=states)
data = data.join(place_data.drop(columns='state'), how='inner')

args = {
    'data': data,
    'place_data' : place_data,
    'use_rw': False,
    'rw_scale': 1e-2,
    'det_noise_scale' : 0.2
}

prob_model = SEIR_hierarchical

kernel = NUTS(prob_model,
              init_strategy = numpyro.infer.util.init_to_median())

mcmc = MCMC(kernel, 
            num_warmup=100,
            num_samples=100,
            num_chains=1)

mcmc.run(jax.random.PRNGKey(1), use_obs=True, **args)

mcmc.print_summary()
mcmc_samples = mcmc.get_samples()

(2, 64)
(2, 64)
(2, 64)
(2, 64)
(2, 64)
(2, 64)
(2, 64)
(2, 64)
(2, 64)
(2, 64)


  0%|          | 0/200 [00:00<?, ?it/s]

(2, 64)
(2, 64)
(2, 64)
(2, 64)
(2, 64)
(2, 64)


sample: 100%|██████████| 200/200 [03:08<00:00,  1.06it/s, 511 steps of size 9.92e-03. acc. prob=0.96] 


(2, 64)
(2, 64)


In [5]:
# Prior samples for comparison
#prior = Predictive(prob_model, posterior_samples = {}, num_samples = 100)
#prior_samples = prior(PRNGKey(2), **args)

# Posterior predictive samples for visualization
args['rw_scale'] = 0 # set drift to zero for forecasting
post_pred = Predictive(prob_model, posterior_samples = mcmc_samples)
post_pred_samples = post_pred(PRNGKey(2), T_future=100, **args)

UnboundLocalError: local variable 'death0' referenced before assignment

In [8]:
util.write_summary('US_covariates', mcmc)
util.save_samples('US_covariates', prior_samples, mcmc_samples, post_pred_samples)

In [4]:
print("gamma", 1/mcmc_samples['I_duration'].mean(axis=0))
print("R0", mcmc_samples['R0'].mean(axis=0))
print("future", post_pred_samples['R0_future'].mean(axis=0))


gamma [0.46590927 0.37561768]
R0 [[4.842176  4.7513676 4.646326  4.565598  4.45997   4.4427485 4.3338637
  4.326357  4.2234273 4.211689  4.074867  4.0203013 3.9820075 3.9192376
  3.9169238 3.8032105 3.762114  3.6894116 3.7324297 3.6778169 2.0468407
  1.9653195 1.9398962 1.8913001 1.7847192 1.6967947 1.6656678 1.6485134
  1.5927757 1.5278186 1.444906  1.4398555 1.3693429 1.3272738 1.3481774
  1.2758585 1.2583234 1.2005403 1.2083038 1.22061   1.2076305 1.1672586
  1.1828222 1.1775874 1.1611384 1.1618078 1.125301  1.1900162 1.1432829
  1.194088  1.2293613 1.258993  1.2550292 1.2412369 1.2525284 1.2982565
  1.2610795 1.2856754 1.3163527 1.3473303 1.3292767 1.3245267 1.3422028
  1.312683 ]
 [4.684515  4.6720967 4.6767344 4.6475043 4.687548  4.7374773 4.7976747
  4.903584  4.986625  5.075269  5.1828046 5.256796  5.3434544 5.4600086
  5.5070057 5.5770593 3.083275  3.024511  3.0212765 2.967299  2.9433374
  2.9333155 2.8808265 2.8667746 2.8334155 2.8428109 2.8649626 2.8287706
  2.880078  2.8591

NameError: name 'post_pred_samples' is not defined

In [ ]:
T = 100
scale = 'log'

places = data.index.unique(level=0)
start = data.index.unique(level=1).min()
num_places = len(places)

for i, place in zip(range(num_places), places):
    
    t = pd.date_range(start=start, periods=T, freq='D')
    
    shared_params = ['beta0_base', 'gamma_base', 'sigma_base', 'det_rate_base']
    place_samples = {k: v[:,i,...] for k, v in post_pred_samples.items() if not k in shared_params}
    
    t = pd.date_range(start=start, periods=T, freq='D')
    
    positive = data.loc[place].positive
    death = data.loc[place].death

    
    fig, ax = plot_forecast(place_samples, T, positive, t=t, scale=scale, death=death)

    #plot_samples(place_samples, plot_fields=['I'], ax=ax, t=t, T=T)
    #plt.plot(obs[i,:], 'o')

    name = place
    plt.suptitle(f'{name} {T} days ')
    plt.tight_layout()
    plt.show()

    
    